# Exploitation de données météorologiques

In [19]:
#Chargement des librairies
import pandas as pd
import requests
import time

In [3]:
#Configuration
#Configuration
srcFolder = "../data/processed/tempo_202510201126/"
dstFolder = "../data/processed/tempo_202510201126/"

In [4]:
#Chargement des données
df = pd.read_csv(srcFolder+"phenologie.csv")
df.head()

,Unnamed: 0,date,annee,jour_de_l_annee,source_donnees,latitude_du_site,longitude_du_site,regne,genre,espece,code_du_stade_phenologique
0,13,1950-03-23,1950,82,AgroClim Pheno,48.8,2.08,Plantae,Acer,platanoides,61
1,14,1950-04-14,1950,104,AgroClim Pheno,48.8,2.08,Plantae,Acer,platanoides,65
2,15,1951-04-03,1951,93,AgroClim Pheno,48.8,2.08,Plantae,Acer,platanoides,61
3,1801,1965-04-13,1965,103,AgroClim Pheno,48.8,2.08,Plantae,Acer,platanoides,65
4,1802,1966-04-06,1966,96,AgroClim Pheno,48.8,2.08,Plantae,Acer,platanoides,61


In [ ]:
#Test sur 10 résultats

def get_gdd_from_open_meteo(lat, lon, date, t_base=5):
    """
    Récupère les températures et calcule GDD cumulé depuis le 1er janvier
    """
    # Extraire l'année
    year = pd.to_datetime(date).year
    start_date = f"{year}-01-01"
    
    # Requête API
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        'latitude': lat,
        'longitude': lon,
        'start_date': start_date,
        'end_date': date,
        'daily': 'temperature_2m_max,temperature_2m_min',
        'timezone': 'auto'
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()  # Lève une exception si erreur HTTP
    data = response.json()
    
    # Calculer GDD
    temps = pd.DataFrame({
        'date': pd.to_datetime(data['daily']['time']),
        't_max': data['daily']['temperature_2m_max'],
        't_min': data['daily']['temperature_2m_min']
    })
    
    temps['t_mean'] = (temps['t_max'] + temps['t_min']) / 2
    temps['gdd_daily'] = (temps['t_mean'] - t_base).clip(lower=0)
    gdd_cumul = temps['gdd_daily'].sum()
    
    return gdd_cumul

# Exemple d'utilisation
lat, lon = 48.8, 2.49
date = "2023-02-01"
#gdd = get_gdd_from_open_meteo(lat, lon, date, t_base=5)
#print(f"GDD cumulé au {date} : {gdd:.1f}°C·jours")

dfTest = df.copy()
dfTest["gdd_5"] = float(0)

ind = 0
batch = 1000

for index, row in dfTest.iloc[ind*batch:(ind+1)*batch].iterrows():
    print(index)

    try:
        dfTest.loc[index, 'gdd_5'] = get_gdd_from_open_meteo(
            row['latitude_du_site'], 
            row['longitude_du_site'], 
            row['date'], 
            print(f"Rate limit atteint à la ligne {index}, pause de 60 secondes...")
            time.sleep(60)
            # Réessayer la requête
            dfTest.loc[index, 'gdd_5'] = get_gdd_from_open_meteo(
                row['latitude_du_site'], 
                row['longitude_du_site'], 
                row['date'], 
                t_base=5
            )
        else:
            raise

    dfTest.loc[index, 'gdd_5'] = get_gdd_from_open_meteo(row['latitude_du_site'], row['longitude_du_site'], row['date'], t_base=5)

dfTest.to_csv(f"{dstFolder}phenologie_gdd5_{ind}.csv")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58


KeyboardInterrupt: 